<a href="https://colab.research.google.com/github/HarshaVardhanLanka/OpenDeepResearchAgent/blob/main/Day_7(Model_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai tavily-python pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 8.3 MB/s eta 0:00:00


In [2]:
import os
import io
import json
import datetime
import google.generativeai as genai
from tavily import TavilyClient
from google.colab import userdata
from google.colab import files
import pypdf
from IPython.display import display, Markdown

# --- API CONFIGURATION ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY')
except:
    print("⚠️ Error: Please set GOOGLE_API_KEY and TAVILY_API_KEY in Colab Secrets.")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro-latest')
tavily = TavilyClient(api_key=TAVILY_API_KEY)
print("Setup complete.")

Setup complete.


In [3]:
# --- MEMORY SYSTEM ---
class HistoryManager:
    def __init__(self):
        self.history_file = "agent_history.json"
        self.history = self.load_history()

    def load_history(self):
        """Loads history from a JSON file if it exists."""
        if os.path.exists(self.history_file):
            with open(self.history_file, 'r') as f:
                return json.load(f)
        return []

    def save_entry(self, input_text, mode, final_report):
        """Saves a new research session to memory."""
        entry = {
            "id": len(self.history) + 1,
            "timestamp": str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")),
            "mode": mode,
            "input": input_text[:15] + "..." if len(input_text) > 50 else input_text, # Preview only
            "full_input": input_text,
            "report": final_report
        }
        self.history.append(entry)
        with open(self.history_file, 'w') as f:
            json.dump(self.history, f)
        print("✅ Research saved to History.")

    def show_history(self):
        """Displays list of past inputs."""
        if not self.history:
            print("\n📭 History is empty.")
            return None

        print("\n📚 RESEARCH HISTORY:")
        print(f"{'ID':<4} | {'Time':<18} | {'Mode':<8} | {'Topic/Input'}")
        print("-" * 60)
        for item in self.history:
            print(f"{item['id']:<4} | {item['timestamp']:<18} | {item['mode']:<8} | {item['input']}")
        return self.history
# Initialize Memory
memory = HistoryManager()

# --- HELPER: PDF TEXT EXTRACTOR ---
def extract_pdf_text(uploaded_file_bytes):
    try:
        pdf_reader = pypdf.PdfReader(io.BytesIO(uploaded_file_bytes))
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {e}"

# --- STRICT AGENT STEPS ---

def step_1_generate_questions(input_text, mode):
    """Generates 3 questions based on mode."""
    print(f"\nStep 1: Generating 3 sub-questions (Mode: {mode})...")

    if mode == "topic":
        prompt = f"""
        Task: Break down the topic "{input_text}" into exactly 3 searchable questions.
        Constraint: Output ONLY the 3 questions separated by newlines. No numbering.
        """
    else: # mode == pdf
        prompt = f"""
        Task: Read the following paper excerpt and generate 3 search questions to VALIDATE the claims or check for newer updates.
        Paper Context: {input_text[:10000]}
        Constraint: Output ONLY the 3 questions separated by newlines. No numbering.
        """

    response = model.generate_content(prompt)
    questions = [q.strip() for q in response.text.strip().split('\n') if q.strip()]
    return questions[:3]

def step_2_get_responses(questions):
    """Retrieves short responses (Max 1 result per question)."""
    print(f"Step 2: Retrieving short responses from the web...")
    q_and_a_results = []

    for i, q in enumerate(questions, 1):
        print(f"   {i}. Searching: {q}")
        try:
            # max_results=1 forces a short response
            search_result = tavily.search(query=q, search_depth="basic", max_results=1)

            if search_result['results']:
                content = search_result['results'][0]['content'][:600] # Limit char count
                q_and_a_results.append(f"Question: {q}\nFound Info: {content}")
            else:
                q_and_a_results.append(f"Question: {q}\nFound Info: No data.")
        except:
             q_and_a_results.append(f"Question: {q}\nFound Info: Search Failed.")

    return "\n---\n".join(q_and_a_results)

def step_3_summarize(input_text, research_data, mode="topic"):
    """Summarizes everything into a final report."""
    print(f"📝 Synthesizing final report...")

    if mode == "topic":
        prompt = f"""
        Write a comprehensive research summary on the topic below, incorporating the search findings.

        User Topic: {input_text}
        Search Data: {research_data}

        Format: Markdown with headers.
        """
    else:
        prompt = f"""
        Create a "Deep Dive" analysis of the provided research paper using the external search data.

        1. Summarize the Paper's core contribution.
        2. Compare it with the "External Validation" findings (State-of-the-art, updates, or contradictions).
        3. Provide a final conclusion on the paper's relevance today.

        Paper Text (Context): {input_text[:15000]}
        External Validation Data: {research_data}

        Format: Markdown with headers.
        """

    response = model.generate_content(prompt)

    if not response.candidates:
        feedback = response.prompt_feedback
        error_message = f"Model did not return any candidates. Finish reason: {feedback.finish_reason}. Safety ratings: {feedback.safety_ratings}"
        print(f"Error: {error_message}")
        return f"**Error generating report:** {error_message}"
    try:
        return response.text
    except ValueError as e:
        feedback = response.prompt_feedback
        error_message = f"Failed to access response.text: {e}. Finish reason: {feedback.finish_reason}. Safety ratings: {feedback.safety_ratings}"
        print(f"Error: {error_message}")
        return f"**Error generating report:** {error_message}"

In [5]:
# --- MAIN LOOP ---

def main():
    while True:
        print("\n" + "="*15 + " STRICT HYBRID AGENT " + "="*15)
        print("1. Research a Topic (Text 💬)")
        print("2. Research a Paper (Upload PDF 📖)")
        print("3. View History 📜")
        print("4. Exit 📤")

        choice = input("Select Option: ")

        if choice == '4':
            print("Goodbye.👋")
            break

        # --- OPTION 3: HISTORY ---
        if choice == '3':
            history_data = memory.show_history()
            if history_data:
                sub_choice = input("\nEnter ID to view full report (or Press Enter to go back): ")
                if sub_choice.isdigit():
                    idx = int(sub_choice) - 1
                    if 0 <= idx < len(history_data):
                        entry = history_data[idx]
                        print(f"\n🔁 RECALLING REPORT FOR: {entry['input']}")
                        print("-" * 50)
                        display(Markdown(entry['report']))
                        input("\nPress Enter to continue...")
            continue # Skip the rest and go back to menu

        # SETUP INPUT
        mode = ""
        input_data = ""
        label = "" # Used for history title

        if choice == '1':
            mode = "topic"
            input_data = input("\nEnter Research Topic: ")
            label = input_data

        elif choice == '2':
            mode = "pdf"
            print("\n⬆️ Upload PDF now:")
            uploaded = files.upload()
            if uploaded:
                filename = next(iter(uploaded))
                input_data = extract_pdf_text(uploaded[filename])
                label = f"PDF: {filename}"
            else:
                print("No file uploaded.")
                continue

        if input_data:
            # --- EXECUTE 3-STEP PROCESS ---

            # 1. Generate Questions
            questions = step_1_generate_questions(input_data, mode)
            print(f"   -> Plan: {questions}")

            # 2. Retrieve Responses
            raw_data = step_2_get_responses(questions)

            # 3. Summarize
            final_report = step_3_summarize(input_data, raw_data, mode)

            print("\n" + "="*20 + " 🎯 FINAL REPORT: " + "="*20 + "\n")
            display(Markdown(final_report))

            # Save to History
            memory.save_entry(label, mode, final_report)

if __name__ == "__main__":
    main()


=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 3

📚 RESEARCH HISTORY:
ID   | Time               | Mode     | Topic/Input
------------------------------------------------------------
1    | 2025-11-27 12:36   | pdf      | PDF: 2511.17546v1.pdf

Enter ID to view full report (or Press Enter to go back): 1

🔁 RECALLING REPORT FOR: PDF: 2511.17546v1.pdf
--------------------------------------------------


Here is a "Deep Dive" analysis of the research paper by Rathi et al. based on the provided text and external validation data.

***

### Deep Dive: Reference Quadrupole Moments from Muonic Atom Lamb Shifts

#### 1. Paper's Core Contribution

The research paper by Rathi et al. proposes a novel experimental method to significantly improve the accuracy of absolute electric quadrupole moments (Q_ref) for light transition elements (Z=23 to 30, including V, Cr, Mn, Co, Ni, Cu, Zn).

The core contribution is the synergistic combination of three key elements to solve a long-standing measurement problem:

1.  **Utilizing Muonic Atoms:** The paper circumvents the primary source of uncertainty in current Q_ref measurements: the complex and often unreliable theoretical calculations of the Electric Field Gradient (EFG) in many-electron atoms. By replacing an electron with a much heavier muon, the system becomes hydrogen-like, making the EFG straightforward to calculate with high accuracy (~1%). Furthermore, the quadrupole interaction is enhanced by a factor of ~10⁷, making the effect much larger and easier to measure.

2.  **Focusing on a Specific Transition:** The authors identify the weakly populated muonic 2s → 2p transition (the Lamb shift) as the ideal target. The hyperfine splitting of the 2p level in this transition is directly proportional to the nuclear quadrupole moment.

3.  **Employing Cryogenic Microcalorimeters (MCs):** This is the key technological enabler. Previous attempts to use muonic atoms in this Z-range were stymied by detector limitations. The 2s → 2p transitions are too weak for low-efficiency crystal spectrometers and their energy splitting is too small to be resolved by conventional solid-state detectors. The paper argues that modern MCs possess the unique combination of high quantum efficiency and excellent energy resolution (~10 eV) required to resolve the hyperfine structure of these transitions in the 10-40 keV range.

The paper presents a detailed feasibility study for ⁶³Cu, including GEANT4 simulations to optimize muon beam momentum and cascade calculations to estimate signal rates. The authors conclude that their proposed method can reduce the uncertainty in Q_ref by up to an order of magnitude within a single day of measurement, providing a crucial benchmark for nuclear physics and quantum chemistry.

#### 2. Comparison with External Validation Findings

The external data provides strong validation and even enhances the prospects of the proposed method.

*   **On Theoretical Foundations:** The first external data point mentions the use of the "Multiconfiguration Dirac-Fock and General Matrix Elements Program (MCDFGME)" and "leading order QED corrections," referencing one of the paper's authors (Indelicato). This directly **confirms** that the theoretical framework used in the paper for calculating muonic energy levels and transition rates is indeed state-of-the-art and aligns with the accepted high-precision computational methods in the field.

*   **On the Problem's Significance:** The second data point, discussing EFG calculations for Antimony clusters, indirectly **reinforces** the paper's premise. It shows that EFG calculation remains an active and system-specific research area. The fact that calculations are still being developed for different elements and configurations underscores the difficulty of achieving high accuracy universally, especially for the challenging open d-shell elements the paper targets. This validates the paper's motivation for pursuing a method that is largely independent of complex EFG calculations in many-electron systems.

*   **On Technological Feasibility (State-of-the-art Update):** The third external data point is the most significant. The paper bases its feasibility calculations on an MC resolution of "≈10 eV". The external finding reports the development of a TES microcalorimeter with an energy resolution of **3.1 - 4 eV FWHM**. This is a major **positive update**.
    *   **Contradiction:** There is no contradiction; rather, this shows rapid progress in the field.
    *   **Update:** The available detector technology is now 2-3 times better than the conservative estimate used in the paper's simulations. This dramatically strengthens the paper's proposal. A detector with ~3 eV resolution would resolve the hyperfine splitting (which is on the order of hundreds of eV, as seen in Table I) with much greater clarity, reducing statistical uncertainty, shortening the required measurement time, and potentially allowing for the measurement of even smaller effects or weaker transitions.

#### 3. Final Conclusion on the Paper's Relevance Today

The paper by Rathi et al. is **highly relevant and forward-looking**. It is not a report of a completed experiment but a compelling and well-researched proposal that charts a clear path to solving a significant problem in fundamental physics.

1.  **Addresses a Critical Gap:** The paper correctly identifies a "gap" in the nuclear chart where reference quadrupole moments for transition metals are poorly known due to the limitations of existing methods. Improving these Q_ref values would have a cascading effect, refining the moments for entire isotopic chains and providing stringent tests for nuclear structure theories that model nuclear deformation.

2.  **Methodologically Sound and Innovative:** The proposed combination of muonic atoms and cryogenic microcalorimeters is innovative and expertly tailored to the problem. It cleverly leverages the unique properties of muons to create a "clean" system for measurement while harnessing the latest advances in detector technology to make that measurement possible.

3.  **Strengthened by Current Technology:** As confirmed by the external data, the technological premise of the paper is not just sound but has been surpassed. The availability of detectors with 3-4 eV resolution makes the proposed experiment more feasible and potentially more precise than the authors originally projected.

In conclusion, this paper serves as an essential blueprint for future experiments at muon facilities like the Paul Scherrer Institute (PSI). Its relevance lies in its potential to unlock a new tier of precision in nuclear moments for a crucial set of elements, thereby providing benchmark data that will be invaluable to both nuclear physicists and quantum chemists for years to come. The work is a prime example of how advances in instrumentation can reopen and solve long-standing fundamental questions.


Press Enter to continue...

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 4
Goodbye.👋
